In [20]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers, Model, Sequential

import numpy as np

from tqdm import tqdm

In [21]:
df = pd.read_csv("./dataset/career_pred.csv")

In [22]:
df.iloc[:49,:-1].columns

Index(['acedamic percentage in operating systems', 'percentage in algorithms',
       'percentage in programming concepts',
       'percentage in software engineering', 'percentage in computer networks',
       'percentage in electronics subjects',
       'percentage in computer architecture', 'percentage in mathematics',
       'percentage in communication skills', 'hours working per day',
       'logical quotient rating', 'hackathons', 'coding skills rating',
       'public speaking points', 'can work long time before system?',
       'self-learning capability?', 'extra-courses did', 'certifications',
       'workshops', 'talenttests taken?', 'olympiads',
       'reading and writing skills', 'memory capability score',
       'interested subjects', 'interested career area', 'job/higher studies?',
       'type of company want to settle in?',
       'taken inputs from seniors or elders', 'interested in games',
       'interested type of books', 'salary range expected',
       'in a real

In [23]:
labels = df["suggested job role"]
labels_unique = labels.unique().tolist()
labels = labels.map(lambda x: labels_unique.index(x))
labels = to_categorical(labels)

In [24]:
df = df[df.columns[:-1]]

In [25]:
k = {}
k["certification"]=df["certifications"].unique().tolist()
k["workshops"]=df["workshops"].unique().tolist()
k["interested subjects"]=df["interested subjects"].unique().tolist()
k["interested career area"]=df["interested career area"].unique().tolist()
k['type of company want to settle in?']=df['type of company want to settle in?'].unique().tolist()

In [26]:
labels_unique

['Database Developer',
 'Portal Administrator',
 'Systems Security Administrator',
 'Business Systems Analyst',
 'Software Systems Engineer',
 'Business Intelligence Analyst',
 'CRM Technical Developer',
 'Mobile Applications Developer',
 'UX Designer',
 'Quality Assurance Associate',
 'Web Developer',
 'Information Security Analyst',
 'CRM Business Analyst',
 'Technical Support',
 'Project Manager',
 'Information Technology Manager',
 'Programmer Analyst',
 'Design & UX',
 'Solutions Architect',
 'Systems Analyst',
 'Network Security Administrator',
 'Data Architect',
 'Software Developer',
 'E-Commerce Analyst',
 'Technical Services/Help Desk/Tech Support',
 'Information Technology Auditor',
 'Database Manager',
 'Applications Developer',
 'Database Administrator',
 'Network Engineer',
 'Software Engineer',
 'Technical Engineer',
 'Network Security Engineer',
 'Software Quality Assurance (QA) / Testing']

In [27]:
df.columns

Index(['acedamic percentage in operating systems', 'percentage in algorithms',
       'percentage in programming concepts',
       'percentage in software engineering', 'percentage in computer networks',
       'percentage in electronics subjects',
       'percentage in computer architecture', 'percentage in mathematics',
       'percentage in communication skills', 'hours working per day',
       'logical quotient rating', 'hackathons', 'coding skills rating',
       'public speaking points', 'can work long time before system?',
       'self-learning capability?', 'extra-courses did', 'certifications',
       'workshops', 'talenttests taken?', 'olympiads',
       'reading and writing skills', 'memory capability score',
       'interested subjects', 'interested career area', 'job/higher studies?',
       'type of company want to settle in?',
       'taken inputs from seniors or elders', 'interested in games',
       'interested type of books', 'salary range expected',
       'in a real

In [28]:
exclusion = [
    'Hours working per day',
    'Logical quotient rating',
    'hackathons',
    'coding skills rating',
    'public speaking points']

items_1 = [
    'Acedamic percentage in Operating Systems', 
    'percentage in Algorithms',
    'Percentage in Programming Concepts',
    'Percentage in Software Engineering', 
    'Percentage in Computer Networks',
    'Percentage in Electronics Subjects',
    'Percentage in Computer Architecture', 
    'Percentage in Mathematics',
    'Percentage in Communication skills']

items_2 = [
    'can work long time before system?',
    'self-learning capability?', 
    'Extra-courses did',
    'Management or Technical', 
    'Salary/work', 
    'hard/smart worker', 
    'worked in teams ever?', 
    'Introvert', 
    'Job/Higher Studies?']

items_3 = [
    'certifications',
    'workshops',
    'reading and writing skills',
    'memory capability score',
    'Interested subjects',
    'interested career area',
    'Type of company want to settle in?']

# convert all the lists above to have lower case strings
exclusion = [i.lower() for i in exclusion]
items_1 = [item.lower() for item in items_1]
items_2 = [item.lower() for item in items_2]
items_3 = [item.lower() for item in items_3]

df = df[items_1+items_2+items_3+exclusion]

print(len(items_1+items_2+items_3+exclusion))

def convert_to_x(row):
    global df

    labels = []

    for col in row.index:
        if col in items_1:
            labels.append(row[col]/100)
        elif col in items_2:
            labels.append(0 if row[col] == "no" else 1)
        elif col in items_3:
            uniques = df[col].unique().tolist()
            labels.append(uniques.index(row[col]))
        elif col in exclusion:
            labels.append(row[col])

    return labels

30


In [29]:
df.iloc[0].index

Index(['acedamic percentage in operating systems', 'percentage in algorithms',
       'percentage in programming concepts',
       'percentage in software engineering', 'percentage in computer networks',
       'percentage in electronics subjects',
       'percentage in computer architecture', 'percentage in mathematics',
       'percentage in communication skills',
       'can work long time before system?', 'self-learning capability?',
       'extra-courses did', 'management or technical', 'salary/work',
       'hard/smart worker', 'worked in teams ever?', 'introvert',
       'job/higher studies?', 'certifications', 'workshops',
       'reading and writing skills', 'memory capability score',
       'interested subjects', 'interested career area',
       'type of company want to settle in?', 'hours working per day',
       'logical quotient rating', 'hackathons', 'coding skills rating',
       'public speaking points'],
      dtype='object')

In [30]:
xs = np.array([convert_to_x(df.iloc[i]) for i in tqdm(df.index)])
np.save("./dataset/xs.npy", xs)

100%|██████████| 20000/20000 [03:15<00:00, 102.46it/s]


In [31]:
xs.shape

(20000, 30)

In [32]:
xs = np.load("./dataset/xs.npy")

In [33]:
def group_list(l, group_size):
    for i in range(0, len(l), group_size):
        yield l[i:i+group_size]

BATCH_SIZE = 100

split_at = int(len(xs)//BATCH_SIZE * 0.8)
dataset = tf.data.Dataset.from_tensor_slices((xs, labels)).shuffle(1000).batch(BATCH_SIZE)

train_dataset = dataset.take(split_at)
test_dataset = dataset.skip(split_at)

In [34]:
sum_items = len(items_1) + len(items_2) + len(items_3) + len(exclusion)

In [35]:
def make_model():
    _input = layers.Input(shape=(sum_items,))

    a, b, c, d = tf.split(_input, [len(items_1), len(items_2), len(items_3), len(exclusion)], axis=1)

    e1 = layers.Dense(32, activation="relu")(a)
    # # e1 = layers.LeakyReLU(0.2)(e1)
    e2 = layers.Dense(32, activation="relu")(b)
    # # e1 = layers.LeakyReLU(0.2)(e1)
    e3 = layers.Dense(32, activation="relu")(c)
    # # e1 = layers.LeakyReLU(0.2)(e1)
    e4 = layers.Dense(32, activation="relu")(d)
    # e4 = layers.LeakyReLU(0.2)(e4)

    e = layers.concatenate([e1, e2, e3, e4])
    # e = layers.Dropout(0.2)(e)
    # e = layers.LayerNormalization()(e)

    # use multihead attention
    attention = layers.MultiHeadAttention(num_heads=8, key_dim=8, attention_axes=(1,))(_input, _input)
    # attention = layers.LayerNormalization()(attention)

    e = layers.Dense(1024, activation="relu")(e)
    e = layers.Dropout(0.2)(e)
    e = layers.Dense(1024, activation="relu")(e)
    e = layers.Dropout(0.2)(e)
    e = layers.Dense(512, activation="relu")(e)
    e = layers.Dropout(0.2)(e)
    e = layers.Dense(256, activation="relu")(e)
    e = layers.Dense(labels.shape[-1], activation="softmax")(e)

    return Model(inputs=[_input], outputs=e)

model = make_model()
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.0001, epsilon=1e-07),
    loss="categorical_crossentropy",
    metrics=["categorical_accuracy"]
)

In [36]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 30)]         0           []                               
                                                                                                  
 tf.split_1 (TFOpLambda)        [(None, 9),          0           ['input_2[0][0]']                
                                 (None, 9),                                                       
                                 (None, 7),                                                       
                                 (None, 5)]                                                       
                                                                                                  
 dense_9 (Dense)                (None, 32)           320         ['tf.split_1[0][0]']       

In [37]:
history = model.fit(
    train_dataset,
    validation_data=test_dataset,
    epochs=1000,
)

Epoch 1/1000
160/160 [==============================] - 3s 16ms/step - loss: 3.5409 - categorical_accuracy: 0.0471 - val_loss: 3.5210 - val_categorical_accuracy: 0.0602
Epoch 2/1000
160/160 [==============================] - 2s 10ms/step - loss: 3.5247 - categorical_accuracy: 0.0543 - val_loss: 3.5201 - val_categorical_accuracy: 0.0615
Epoch 3/1000
160/160 [==============================] - 2s 12ms/step - loss: 3.5224 - categorical_accuracy: 0.0540 - val_loss: 3.5197 - val_categorical_accuracy: 0.0602
Epoch 4/1000
160/160 [==============================] - 2s 10ms/step - loss: 3.5200 - categorical_accuracy: 0.0544 - val_loss: 3.5201 - val_categorical_accuracy: 0.0602
Epoch 5/1000
160/160 [==============================] - 2s 11ms/step - loss: 3.5199 - categorical_accuracy: 0.0542 - val_loss: 3.5182 - val_categorical_accuracy: 0.0615
Epoch 6/1000
160/160 [==============================] - 2s 10ms/step - loss: 3.5186 - categorical_accuracy: 0.0542 - val_loss: 3.5173 - val_categorical_acc

KeyboardInterrupt: 

In [38]:
model.save("./models/main.h5")

In [ ]:
x = [
    0.82,0.81,0.85,0.78,0.80,0.70,0.80,0.91,0.81,
    10,8,3,8,6,
    1,1,1,
    1,3,0,1,
    2,2,7,2,
    0,8,0,1,29,0,
    1,1,1,1,1,1,1
]

In [ ]:
(model(np.expand_dims(np.array(x), axis=0)).numpy() * 100)[0].tolist()

In [ ]:
labels_unique[0]

In [ ]:
test = np.expand_dims(np.array(convert_to_x(df.iloc[1000])), axis=0)
labels[1000].tolist().index(1)

In [ ]:
np.argmax(model(test))

In [ ]:
df["interested career area "].unique()

In [ ]:
labels_unique